ValueError: not enough values to unpack (expected 3, got 0)

In [5]:
import cv2
import numpy as np
image_path=r"D:\\pcbprojecta\\BACKUP\\data\\Mis align component\\0417.jpg"
# Load image
def load_image(image_path):
    image = cv2.imread(image_path)
    return image

# Extract Red, Green, Blue channels
def extract_color_channels(image):
    # Split the image into its RGB channels
    (B, G, R) = cv2.split(image)
    return R, G, B

# Feature Extraction: Location Feature
def extract_location_feature(solder_joints, search_window):
    # Assuming solder_joints is a list of (x, y) coordinates
    x_coords = [joint[0] for joint in solder_joints]
    y_coords = [joint[1] for joint in solder_joints]
    
    # Calculate the mean position of solder joints
    mean_x = np.mean(x_coords)
    mean_y = np.mean(y_coords)
    
    # Calculate the distance from the center of the search window
    distance = np.sqrt((mean_x - search_window[0])**2 + (mean_y - search_window[1])**2)
    return distance

# Feature Extraction: Shape Feature
def extract_shape_feature(solder_image):
    # Thresholding to detect solder joint shape (for simplicity, assume binary image)
    gray = cv2.cvtColor(solder_image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    # Find contours of the solder joint
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Extract shape features such as area, perimeter, and aspect ratio
    contour = contours[0]  # Assuming one contour per solder joint
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    aspect_ratio = float(cv2.boundingRect(contour)[2]) / cv2.boundingRect(contour)[3]
    
    return area, perimeter, aspect_ratio

# Feature Extraction: Color Feature
def extract_color_feature(R, G, B, solder_area):
    # Calculate the average color intensity within the solder area
    red_mean = np.mean(R[solder_area])
    green_mean = np.mean(G[solder_area])
    blue_mean = np.mean(B[solder_area])
    
    return red_mean, green_mean, blue_mean

# Rule-based Inspection Criterion
def inspection_criterion(features, threshold):
    # Example: if area is too small or large, classify as defective
    area, perimeter, aspect_ratio = features
    if area < threshold['min_area'] or area > threshold['max_area']:
        return "Defective"
    if perimeter < threshold['min_perimeter']:
        return "Defective"
    
    # Add more rules as needed
    return "Acceptable"

# Main function to process the image and extract features
def process_image(image_path, solder_joints, search_window, threshold):
    # Load and process the image
    image = load_image(image_path)
    
    # Extract color channels
    R, G, B = extract_color_channels(image)
    
    # Extract location feature
    location_feature = extract_location_feature(solder_joints, search_window)
    
    # Extract shape feature
    shape_feature = extract_shape_feature(image)
    
    # Extract color feature (assuming a region of interest, e.g., solder area)
    color_feature = extract_color_feature(R, G, B, solder_joints)
    
    # Combine all features into a feature vector
    features = shape_feature + color_feature
    
    # Apply inspection criterion
    result = inspection_criterion(features, threshold)
    
    return result

# Example usage
solder_joints = [(100, 200), (150, 250), (200, 300)]  # Example solder joint positions
search_window = (120, 220)  # Example search window center (x, y)
threshold = {'min_area': 50, 'max_area': 1000, 'min_perimeter': 50}

result = process_image('solder_image.jpg', solder_joints, search_window, threshold)
print(result)

ValueError: not enough values to unpack (expected 3, got 0)

In [12]:
import os
os.chdir("C:\\Users\\jadon\\Desktop\\total folder\\yolov5")

In [11]:
import customtkinter as ctk
from tkinter import messagebox
import serial
import time
import re


class CNCController:
    def __init__(self, master):
        self.master = master
        self.master.title("CNC Controller")
        self.master.geometry("800x600")
        ctk.set_appearance_mode("System")
        ctk.set_default_color_theme("blue")

        # Serial connection setup
        self.ser = None
        self.baud_rate = 250000  # Default baud rate
        self.feedrate = 1000  # Default feedrate

        # Setup sections
        self.setup_original_section()
        self.setup_axis_controls()
        self.setup_console_output()

    def setup_original_section(self):
        original_frame = ctk.CTkFrame(self.master)
        original_frame.pack(side="left", fill="y", padx=10, pady=10)

        # Firmware Selection
        self.firmware_label = ctk.CTkLabel(original_frame, text="Select Firmware:")
        self.firmware_label.pack(pady=5)
        self.firmware_dropdown = ctk.CTkOptionMenu(original_frame, values=["GRBL", "Marlin"], command=self.set_firmware)
        self.firmware_dropdown.set("Marlin")
        self.firmware_dropdown.pack(pady=5)

        # Port Selection
        self.port_label = ctk.CTkLabel(original_frame, text="Enter Port (e.g., COM3):")
        self.port_label.pack(pady=5)
        self.port_entry = ctk.CTkEntry(original_frame, width=200)
        self.port_entry.pack(pady=5)

        # Baud Rate Selection
        self.baud_label = ctk.CTkLabel(original_frame, text="Select Baud Rate:")
        self.baud_label.pack(pady=5)
        self.baud_dropdown = ctk.CTkOptionMenu(original_frame, values=["9600", "115200", "250000"], command=self.set_baud_rate)
        self.baud_dropdown.set("250000")
        self.baud_dropdown.pack(pady=5)

        # Connect and Disconnect Buttons
        self.connect_button = ctk.CTkButton(original_frame, text="Connect", command=self.connect_to_cnc)
        self.connect_button.pack(pady=10)
        self.disconnect_button = ctk.CTkButton(original_frame, text="Disconnect", command=self.disconnect_from_cnc)
        self.disconnect_button.pack(pady=10)

        # Feedrate Control
        self.feedrate_label = ctk.CTkLabel(original_frame, text="Feedrate:")
        self.feedrate_label.pack(pady=10)

        self.feedrate_slider = ctk.CTkSlider(original_frame, from_=100, to=5000, number_of_steps=49)
        self.feedrate_slider.set(1000)  # Default feedrate
        self.feedrate_slider.pack(pady=10)

        # STOP Button
        self.stop_button = ctk.CTkButton(original_frame, text="STOP", fg_color="red", command=self.stop_motion)
        self.stop_button.pack(pady=10)

    def setup_axis_controls(self):
        axises = ctk.CTkFrame(self.master, corner_radius=10)
        axises.pack(side="top", anchor="nw", pady=10, padx=10, fill="x")
        axis_frame = ctk.CTkFrame(axises, corner_radius=10)
        axis_frame.pack(side="left", anchor="ne", pady=10, padx=10, fill="x")

        self.x_label = ctk.CTkLabel(axis_frame, text="Input X Value:")
        self.x_label.pack(anchor="nw", pady=5)
        self.x_entry = ctk.CTkEntry(axis_frame, width=140)
        self.x_entry.pack(anchor="nw", pady=5)

        self.y_label = ctk.CTkLabel(axis_frame, text="Input Y Value:")
        self.y_label.pack(anchor="nw", pady=5)
        self.y_entry = ctk.CTkEntry(axis_frame, width=140)
        self.y_entry.pack(anchor="nw", pady=5)

        self.z_label = ctk.CTkLabel(axis_frame, text="Input Z Value:")
        self.z_label.pack(anchor="nw", pady=5)
        self.z_entry = ctk.CTkEntry(axis_frame, width=140)
        self.z_entry.pack(anchor="nw", pady=5)

        self.set_values_button = ctk.CTkButton(axis_frame, text="Set Values", command=self.set_axis_values)
        self.set_values_button.pack(anchor="nw", pady=10)
        self.refresh_button = ctk.CTkButton(axis_frame, text="Refresh", command=self.refresh_values)
        self.refresh_button.pack(anchor="nw", pady=10)
        self.apply_motion_button = ctk.CTkButton(axis_frame, text="Apply Motion", command=self.apply_motion)
        self.apply_motion_button.pack(anchor="nw", pady=10)

        # Movement Controls
        self.move_x_positive_button = ctk.CTkButton(axis_frame, text="Move X +", command=lambda: self.move_axis('X', 10))
        self.move_x_positive_button.pack(pady=5)

        self.move_x_negative_button = ctk.CTkButton(axis_frame, text="Move X -", command=lambda: self.move_axis('X', -10))
        self.move_x_negative_button.pack(pady=5)

        self.move_y_positive_button = ctk.CTkButton(axis_frame, text="Move Y +", command=lambda: self.move_axis('Y', 10))
        self.move_y_positive_button.pack(pady=5)

        self.move_y_negative_button = ctk.CTkButton(axis_frame, text="Move Y -", command=lambda: self.move_axis('Y', -10))
        self.move_y_negative_button.pack(pady=5)

        motion = ctk.CTkFrame(axises, corner_radius=10)
        motion.pack(side="right", anchor="nw", pady=10, padx=10, fill="x")

        # Additional Features
        self.set_home_button = ctk.CTkButton(motion, text="Set Home", command=self.set_home)
        self.set_home_button.pack(anchor="ne", pady=5)
        self.set_absolute_origin_button = ctk.CTkButton(motion, text="Set Absolute Origin", command=self.set_absolute_origin)
        self.set_absolute_origin_button.pack(anchor="ne", pady=5)
        self.scan_fiducials_button = ctk.CTkButton(motion, text="Scan Fiducials", command=self.scan_fiducials)
        self.scan_fiducials_button.pack(anchor="ne", pady=5)

        self.gcode_label = ctk.CTkLabel(motion, text="Enter G-code:")
        self.gcode_label.pack(anchor="ne", pady=5)
        self.gcode_entry = ctk.CTkEntry(motion, width=140)
        self.gcode_entry.pack(anchor="ne", pady=5)
        self.send_gcode_button = ctk.CTkButton(motion, text="Send G-code", command=self.send_gcode)
        self.send_gcode_button.pack(anchor="nw", pady=10)


    def setup_console_output(self):
        console_frame = ctk.CTkFrame(self.master)
        console_frame.pack(side="bottom", pady=10, padx=10, fill="both", expand=True)

        self.console_label = ctk.CTkLabel(console_frame, text="Console:")
        self.console_label.pack(anchor="nw", padx=5, pady=5)
        self.console_text = ctk.CTkTextbox(console_frame, height=100, width=600)
        self.console_text.pack(padx=5, pady=5, fill="both", expand=True)
        
        
    # Firmware and Baud Rate
    def set_firmware(self, firmware):
        self.console_text.insert("end", f"Firmware set to {firmware}\n")

    def set_baud_rate(self, baud_rate):
        self.baud_rate = int(baud_rate)
        self.console_text.insert("end", f"Baud rate set to {baud_rate}\n")

    # Connection Methods
    def connect_to_cnc(self):
        port = self.port_entry.get()
        if not port or not self.baud_rate:
            messagebox.showerror("Connection Error", "Please specify a port and baud rate.")
            return

        try:
            self.ser = serial.Serial(port, self.baud_rate, timeout=1)
            time.sleep(2)  # Wait for the connection to stabilize
            self.ser.flushInput()
            self.console_text.insert("end", f"Connected to {port} at {self.baud_rate} baud.\n")
        except serial.SerialException as e:
            messagebox.showerror("Connection Error", f"Failed to connect to {port}: {e}")

    def disconnect_from_cnc(self):
        if self.ser and self.ser.is_open:
            self.ser.close()
            self.console_text.insert("end", "Disconnected from CNC.\n")
        else:
            self.console_text.insert("end", "No active connection to disconnect.\n")

    # Feedrate Control
    def update_feedrate(self):
        self.feedrate = int(self.feedrate_slider.get())
        self.console_text.insert("end", f"Feedrate updated to {self.feedrate} mm/min.\n")

    # Axis Controls
    def set_axis_values(self):
        x = self.x_entry.get()
        y = self.y_entry.get()
        z = self.z_entry.get()
        self.console_text.insert("end", f"Set axis values - X: {x}, Y: {y}, Z: {z}\n")

    def refresh_values(self):
        self.x_entry.delete(0, "end")
        self.y_entry.delete(0, "end")
        self.z_entry.delete(0, "end")
        self.console_text.insert("end", "Axis values refreshed.\n")

    def apply_motion(self):
        x =self.x_entry.get()
        y =self.y_entry.get()
        z =self.z_entry.get()
        self.move_axis('X', int(x))  # Convert the string to an integer
        self.move_axis('Y', int(y))  # Convert the string to an integer
        self.move_axis('Z', int(z))
        self.console_text.insert("end", f"Applying motion - X: {x}, Y: {y}, Z: {z}\n")

    def move_axis(self, axis, distance):
        feedrate = int(self.feedrate_slider.get())
        if self.ser and self.ser.is_open:
            command = f"G91\nG1 {axis}{distance} F{feedrate}\nG90\n".encode()
            self.ser.write(command)
            time.sleep(1)
        else:
            messagebox.showerror("Serial Connection Error", "Serial port is not open.")

    # Motion Controls
    def stop_motion(self):
        if self.ser and self.ser.is_open:
            self.ser.write(b"!\n")  # Stop command for GRBL
            self.console_text.insert("end", "Motion stopped.\n")
        else:
            self.console_text.insert("end", "No active connection to stop motion.\n")

    def home_axes(self):
        if self.ser and self.ser.is_open:
            self.ser.write(b'G28 X Y\n')  # Homing command for X and Y axes
            time.sleep(1)  # Adjust based on your CNC's homing speed
        else:
            messagebox.showerror("Serial Connection Error", "Serial port is not open.")

    def set_home(self):
        self.console_text.insert("end", "Setting home...\n")
        self.home_axes()

    def set_absolute_origin(self):
        self.console_text.insert("end", "Setting absolute origin...\n")
        self.ser.write(b"G92 X0 Y0 Z0\n")  # Set absolute origin command

    def scan_fiducials(self):
        self.console_text.insert("end", "Scanning fiducials...\n")



    def send_gcode(self):
    # Get the G-code from the entry field
     gcode = self.gcode_entry.get()

    # Check if the serial connection is open
     if self.ser and self.ser.is_open:
        try:
            # Send the G-code over serial
            self.ser.write((gcode + "\n").encode())
            time.sleep(0.1)  # Short delay to ensure the device processes the command
            response = self.ser.readline().decode().strip()

            # Display the sent G-code and response in the console
            self.console_text.insert("end", f"G-code sent: {gcode}\n")
            self.console_text.insert("end", f"Response: {response}\n")

            # If it's an M-code or motion command, fetch and display coordinates
            if gcode.startswith("M") or gcode.startswith("G"):
                self.ser.write("M114\n".encode())  # Example command to get coordinates
                time.sleep(0.1)  # Allow time for the device to respond
                coordinates = self.ser.readline().decode().strip()

                # Display coordinates in the console and as a message box
                self.console_text.insert("end", f"Coordinates: {coordinates}\n")
                messagebox.showinfo("Coordinates", f"Coordinates are: {coordinates}")
        except Exception as e:
            # Show error if an exception occurs
            messagebox.showerror("Communication Error", f"An error occurred: {e}")
     else:
        # Show an error if the serial port is not open
        messagebox.showerror("Serial Connection Error", "Serial port is not open.")

  



root = ctk.CTk()
cnc_controller = CNCController(root)
root.mainloop()


KeyboardInterrupt: 

In [14]:
import tkinter as tk
from tkinter import scrolledtext
import serial
import threading

class CNCControllerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("CNC Controller")

        self.serial_conn = None
        self.port_var = tk.StringVar()
        self.baudrate_var = tk.StringVar(value="115200")

        self.create_widgets()

    def create_widgets(self):
        tk.Label(self.root, text="Serial Port:").grid(row=0, column=0, padx=5, pady=5)
        self.port_entry = tk.Entry(self.root, textvariable=self.port_var)
        self.port_entry.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(self.root, text="Baudrate:").grid(row=1, column=0, padx=5, pady=5)
        self.baudrate_entry = tk.Entry(self.root, textvariable=self.baudrate_var)
        self.baudrate_entry.grid(row=1, column=1, padx=5, pady=5)

        self.connect_button = tk.Button(self.root, text="Connect", command=self.connect_serial)
        self.connect_button.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

        self.command_entry = tk.Entry(self.root)
        self.command_entry.grid(row=3, column=0, columnspan=2, padx=5, pady=5)
        self.send_button = tk.Button(self.root, text="Send G-code", command=self.send_gcode)
        self.send_button.grid(row=4, column=0, columnspan=2, padx=5, pady=5)

        self.output_text = scrolledtext.ScrolledText(self.root, width=50, height=10)
        self.output_text.grid(row=5, column=0, columnspan=2, padx=5, pady=5)

        self.disconnect_button = tk.Button(self.root, text="Disconnect", command=self.disconnect_serial)
        self.disconnect_button.grid(row=6, column=0, columnspan=2, padx=5, pady=5)

    def connect_serial(self):
        port = self.port_var.get()
        baudrate = self.baudrate_var.get()
        try:
            self.serial_conn = serial.Serial(port, baudrate, timeout=1)
            self.output_text.insert(tk.END, f"Connected to {port} at {baudrate} baud\n")
        except serial.SerialException as e:
            self.output_text.insert(tk.END, f"Connection Error: {e}\n")


    def send_gcode(self):
        if self.serial_conn and self.serial_conn.is_open:
            gcode = self.command_entry.get().strip()
            if gcode:
                self.serial_conn.write((gcode + "\n").encode())
                response = self.serial_conn.readline().decode().strip()
                self.output_text.insert(tk.END, f"> {gcode}\n< {response}\n")
        else:
            self.output_text.insert(tk.END, "Serial connection not established\n")

    def disconnect_serial(self):
        if self.serial_conn:
            self.serial_conn.close()
            self.output_text.insert(tk.END, "Disconnected\n")

if __name__ == "__main__":
    root = tk.Tk()
    app = CNCControllerApp(root)
    root.mainloop()

    


KeyboardInterrupt: 

In [9]:
import tkinter as tk
from tkinter import scrolledtext
import serial
import threading

class CNCControllerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("CNC Controller")

        self.serial_conn = None
        self.port_var = tk.StringVar(value="COM3")  # Default to COM3
        self.baudrate_var = tk.StringVar(value="250000")  # Default to 250000 baud

        self.create_widgets()

    def create_widgets(self):
        tk.Label(self.root, text="Serial Port:").grid(row=0, column=0, padx=5, pady=5)
        self.port_entry = tk.Entry(self.root, textvariable=self.port_var)
        self.port_entry.grid(row=0, column=1, padx=5, pady=5)

        tk.Label(self.root, text="Baudrate:").grid(row=1, column=0, padx=5, pady=5)
        self.baudrate_entry = tk.Entry(self.root, textvariable=self.baudrate_var)
        self.baudrate_entry.grid(row=1, column=1, padx=5, pady=5)

        self.connect_button = tk.Button(self.root, text="Connect", command=self.connect_serial)
        self.connect_button.grid(row=2, column=0, columnspan=2, padx=5, pady=5)

        self.command_entry = tk.Entry(self.root)
        self.command_entry.grid(row=3, column=0, columnspan=2, padx=5, pady=5)
        self.send_button = tk.Button(self.root, text="Send G-code", command=self.send_gcode)
        self.send_button.grid(row=4, column=0, columnspan=2, padx=5, pady=5)

        self.output_text = scrolledtext.ScrolledText(self.root, width=50, height=10)
        self.output_text.grid(row=5, column=0, columnspan=2, padx=5, pady=5)

        self.disconnect_button = tk.Button(self.root, text="Disconnect", command=self.disconnect_serial)
        self.disconnect_button.grid(row=6, column=0, columnspan=2, padx=5, pady=5)

    def connect_serial(self):
        port = self.port_var.get()
        baudrate = self.baudrate_var.get()
        try:
            self.serial_conn = serial.Serial(port, baudrate, timeout=1)
            self.output_text.insert(tk.END, f"Connected to {port} at {baudrate} baud\n")
        except serial.SerialException as e:
            self.output_text.insert(tk.END, f"Connection Error: {e}\n")

    def send_gcode(self):
        if self.serial_conn and self.serial_conn.is_open:
            gcode = self.command_entry.get().strip()
            if gcode:
                self.serial_conn.write((gcode + "\n").encode())
                response = self.serial_conn.readline().decode().strip()
                self.output_text.insert(tk.END, f"> {gcode}\n< {response}\n")
        else:
            self.output_text.insert(tk.END, "Serial connection not established\n")

    def disconnect_serial(self):
        if self.serial_conn:
            self.serial_conn.close()
            self.output_text.insert(tk.END, "Disconnected\n")

if __name__ == "__main__":
    root = tk.Tk()
    app = CNCControllerApp(root)
    root.mainloop()


In [3]:
pip install pyserial

Note: you may need to restart the kernel to use updated packages.


In [7]:
import customtkinter as ctk
from tkinter import messagebox
import serial
import time


class CNCControlApp:
    def __init__(self, master):
        self.master = master
        master.title("CNC Controller")
        master.geometry("600x400")
        ctk.set_appearance_mode("System")
        ctk.set_default_color_theme("blue")

        # Serial connection setup
        self.serial_port = 'COM3'  # The serial port the CNC is connected to
        self.baud_rate = 250000  # Communication speed, must match CNC machine's configuration
        try:
            self.ser = serial.Serial(self.serial_port, self.baud_rate, timeout=1)
            time.sleep(2)  # Wait for the connection to stabilize
            self.ser.write(b'\n\n')  # Wake up command
            time.sleep(2)  # Wait for the CNC to wake up
            self.ser.flushInput()  # Clear any startup text
        except serial.SerialException as e:
            messagebox.showerror("Serial Connection Error", f"Failed to connect to {self.serial_port}: {e}")
            self.master.destroy()

        # Feedrate control UI
        self.feedrate_label = ctk.CTkLabel(master, text="Feedrate:")
        self.feedrate_label.pack(pady=10)

        self.feedrate_slider = ctk.CTkSlider(master, from_=100, to=5000, number_of_steps=49)
        self.feedrate_slider.set(1000)  # Default feedrate
        self.feedrate_slider.pack(pady=10)

        # Movement command buttons
        self.move_x_positive_button = ctk.CTkButton(master, text="Move X +", command=lambda: self.move_axis('X', 10))
        self.move_x_positive_button.pack(pady=5)

        self.move_x_negative_button = ctk.CTkButton(master, text="Move X -", command=lambda: self.move_axis('X', -10))
        self.move_x_negative_button.pack(pady=5)

        self.move_y_positive_button = ctk.CTkButton(master, text="Move Y +", command=lambda: self.move_axis('Y', 10))
        self.move_y_positive_button.pack(pady=5)

        self.move_y_negative_button = ctk.CTkButton(master, text="Move Y -", command=lambda: self.move_axis('Y', -10))
        self.move_y_negative_button.pack(pady=5)

        # Homing button for X and Y axes
        self.home_xy_button = ctk.CTkButton(master, text="Home X & Y Axes", command=self.home_axes)
        self.home_xy_button.pack(pady=10)

    def move_axis(self, axis, distance):
        feedrate = int(self.feedrate_slider.get())
        if self.ser.is_open:
            command = f"G91\nG1 {axis}{distance} F{feedrate}\nG90\n".encode()
            self.ser.write(command)
            time.sleep(1)
        else:
            messagebox.showerror("Serial Connection Error", "Serial port is not open.")

    def home_axes(self):
        if self.ser.is_open:
            self.ser.write(b'G28 X Y\n')  # Homing command for X and Y axes
            time.sleep(1)  # Adjust based on your CNC's homing speed
        else:
            messagebox.showerror("Serial Connection Error", "Serial port is not open.")

    def __del__(self):
        if hasattr(self, 'ser') and self.ser.is_open:
            self.ser.close()


if __name__ == "__main__":
    root = ctk.CTk()
    app = CNCControlApp(root)
    root.mainloop()


TclError: can't invoke "frame" command: application has been destroyed